# Examples

In [ ]:
from linearmodels.datasets import french
data = french.load()
print(french.DESCR)

## 1-Step Estimation using Seemingly Unrelated Regression (SUR)

In [ ]:
from linearmodels.asset_pricing import TradedFactorModel
portfolios = data[['S1V1','S1V3','S1V5','S3V1','S3V3','S3V5','S5V1','S5V3','S5V5']]
factors = data[['MktRF']]
mod = TradedFactorModel(portfolios, factors)
res = mod.fit()
print(res)

In [ ]:
factors = data[['MktRF', 'SMB', 'HML']]
mod = TradedFactorModel(portfolios, factors)
res = mod.fit()
print(res)

In [ ]:
indu = ['NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 
        'Telcm', 'Utils', 'Shops',  'Hlth', 'Money',  'Other']
portfolios = data[indu]
mod = TradedFactorModel(portfolios, factors)
res = mod.fit()
print(res)

## 2-Step Estimation

In [ ]:
from linearmodels.asset_pricing import LinearFactorModel
factors = data[['MktRF', 'SMB', 'HML', 'Mom']]
portfolios = data[['S1M1','S1M3','S1M5','S3M1','S3M3','S3M5','S5M1','S5M3','S5M5']]
mod = LinearFactorModel(portfolios, factors)
print(mod.fit())

In [ ]:
from linearmodels.asset_pricing import LinearFactorModel
factors = data[['MktRF', 'HML', 'Mom']]
portfolios = data[['S1M1','S1M3','S1M5','S3M1','S3M3','S3M5','S5M1','S5M3','S5M5']]
mod = LinearFactorModel(portfolios, factors)
print(mod.fit())

In [ ]:
from linearmodels.asset_pricing import LinearFactorModel
factors = data[['MktRF', 'HML', 'Mom']]
portfolios = data[['S1M1','S1M3','S1M5','S3M1','S3M3','S3M5','S5M1','S5M3','S5M5']]
mod = LinearFactorModel(portfolios, factors, risk_free=True)
print(mod.fit())

In [ ]:
mod = LinearFactorModel(portfolios, factors)
print(mod.fit(cov_type='kernel'))

## GMM Estimation

In [ ]:
from linearmodels.asset_pricing import LinearFactorModelGMM
mod = LinearFactorModelGMM(portfolios, factors)
res = mod.fit()
print(res)

In [ ]:
res = mod.fit(cov_type='kernel', kernel='bartlett')
print(res)

In [ ]:
res = mod.fit(use_cue=True)
print(res)

In [ ]:
res = mod.fit(steps=10, disp=25)
print(res)